# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
#pip install gmaps


In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,12.65,68,58,1.79,PT,1680493777
1,1,port alfred,-33.5906,26.8910,18.43,88,100,0.89,ZA,1680493778
2,2,ushuaia,-54.8000,-68.3000,11.81,43,20,3.60,AR,1680493778
3,3,san pedro,14.3500,121.0167,33.51,58,20,1.54,PH,1680493778
4,4,hasaki,35.7333,140.8333,14.22,53,21,9.31,JP,1680493484


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
#geoapify.configure(api_key=geoapify_key)
city_data_df["Humidity"].max()

# Display the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map plot
map_plot_1

# Display the map plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_vacay_weather = city_data_df.loc[(city_data_df['Max Temp'] < 78) & (city_data_df['Max Temp'] > 65) \
                                         & (city_data_df['Humidity'] < 50) \
                                         & (city_data_df['Cloudiness'] < 30) \
                                         & (city_data_df['Wind Speed'] < 15)]

# Drop any rows with null values
ideal_vacay_weather = ideal_vacay_weather.dropna()
ideal_vacay_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacay_weather[['City', 'Country', 'Lat', 'Lng']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
hotel_df.head()



,City,Country,Lat,Lng,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# Set parameters to search for a hotel
radius = 5000
params = {
    'types': 'lodging',
    'key': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f'{lat},{lng}'
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["filter"] = # YOUR CODE HERE
    # params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)